In [64]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
from itertools import product
import sympy as sym


%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.4f}'.format})
GHz = 1e9

In [170]:
E_r = sq_ext.diag(sq_ext.sq_resonator().hamiltonian(),4, out='GHz', remove_ground=True)[0]
E_f = sq_ext.diag(sq_ext.sq_fluxonium().hamiltonian(),4, out='GHz', remove_ground=True)[0]
E_q = sq_ext.diag(sq_ext.hamiltonian_qubit(nmax_r=5, nmax_f=10), 6, out='GHz', remove_ground=True)[0]
E_q_C_q = sq_ext.diag(sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=10, Cc=0), 8, out='GHz', remove_ground=True)[0]

In [171]:
sq_ext.generate_and_prioritize_energies([E_f, E_r], 5)

(array([0.0000, 1.9455, 6.1613, 7.2084, 9.1539]),
 [[0, 0], [1, 0], [2, 0], [0, 1], [1, 1]])

In [53]:
E_q

array([0.0000, 1.9452, 6.1597, 7.2085, 9.1538, 10.7891])

In [54]:
E_q_C_q

array([0.0000, 1.9452, 1.9452, 3.8905, 6.1597, 6.1597, 7.2085, 7.2085])

In [1182]:
fluxonium = sq_ext.sq_fluxonium()
H_0_f = fluxonium.hamiltonian() 
H_0 = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f ) #+ 0.00001*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))
H = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f  )+ 1e5*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))

In [1266]:
E, ψ_0_f = np.linalg.eigh(H_0_f.__array__() )
# E, ψ_0_f = sp.sparse.linalg.eigsh(H_0_f.data, 2, which='SA')
E, ψ_0_f = sq_ext.eigs_sorted(E, ψ_0_f)
ψ_0 = np.array([np.kron(ψ_0_f[:,0],ψ_0_f[:,0]), np.kron(ψ_0_f[:,1],ψ_0_f[:,0]), np.kron(ψ_0_f[:,0],ψ_0_f[:,1]), np.kron(ψ_0_f[:,1],ψ_0_f[:,1]) ]).T

In [1219]:
E, ψ_0 = sp.sparse.linalg.eigsh(H_0.data, 2, which='SA')
E, ψ_0 = sq_ext.eigs_sorted(E, ψ_0)
ψ_0 = np.linalg.qr(ψ_0)[0]

In [1256]:
E, ψ_0 = np.linalg.eigh(H_0.__array__())
ψ_0 =  ψ_0[:,:4]

In [1262]:
np.round(ψ_0.T.conj() @ ψ_0,4)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 0.-0.j, 1.+0.j, 0.+0.j],
       [0.-0.j, 0.-0.j, 0.-0.j, 1.+0.j]])

In [1267]:
H_eff = ψ_0.T.conj() @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.9727+0j)	*	 I ⨂σz 
 (0.0235+0j)	*	 σx ⨂σx 
 (-0.9727+0j)	*	 σz ⨂I 


In [1161]:
H_eff = sq_ext.H_eff_p1(H_0, H, out= 'GHz', n_eig=4, solver='numpy')
# H_eff = sq_ext.H_eff_p1(H_0, H, out= 'GHz', n_eig=4)
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.961+0j)	*	 I ⨂σz 
 (0.0125+0j)	*	 σx ⨂σx 
 (-0.011+0j)	*	 σy ⨂σy 
 (-0.9845+0j)	*	 σz ⨂I 


In [1106]:
ψ_0[:,2] /= np.exp(1j*0.1)
H_eff = ψ_0.conj().T @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

ValueError: shapes (4,4) and (625,625) not aligned: 4 (dim 1) != 625 (dim 0)

In [173]:
importlib.reload(sq_ext)

<module 'Modules.SQcircuit_extensions' from 'C:\\Users\\gabri\\PycharmProjects\\tunable-capacitive-coupling-flux-qubits\\Modules\\SQcircuit_extensions.py'>

In [152]:
H, ψ_0 = sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=5, Cc=1,return_Ψ_nonint=True)
# sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=5, Cc=1)

In [155]:
H_eff = sq_ext.H_eff_p1(ψ_0, H, 4, out='GHz', real=False, remove_ground = False)
sq_ext.decomposition_in_pauli_4x4(H_eff, 5);

 (15.2545+0j)	*	 I ⨂I 
 (-0.92634+0j)	*	 I ⨂σz 
 (0.01592+0j)	*	 σy ⨂σy 
 (-0.92634+0j)	*	 σz ⨂I 


In [182]:
H, ψ_0 = sq_ext.hamiltonian_qubit_C_qubit_C_qubit(nmax_r=5, nmax_f=10, Cc=1,return_Ψ_nonint=True, n_eig_Ψ_nonint=8)
# sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=5, Cc=1)

In [187]:
H_eff = sq_ext.H_eff_p1(ψ_0, H, 8, out='GHz', real=False, remove_ground = True)
sq_ext.decomposition_in_pauli_8x8(H_eff, 5);

 (2.8645+0j)	*	 I ⨂ I ⨂ I 
 (0.0086+0j)	*	 I ⨂ I ⨂ σx 
 (-0.47742+0j)	*	 I ⨂ I ⨂ σz 
 (0.0086+0j)	*	 I ⨂ σx ⨂ I 
 (0.0086+0j)	*	 I ⨂ σx ⨂ σx 
 (0.0086+0j)	*	 I ⨂ σy ⨂ σy 
 (-0.47742+0j)	*	 I ⨂ σz ⨂ I 
 (-0.0086+0j)	*	 σx ⨂ I ⨂ I 
 (-0.0086+0j)	*	 σx ⨂ I ⨂ σx 
 (-0.0086+0j)	*	 σx ⨂ σx ⨂ I 
 (-0.0086+0j)	*	 σx ⨂ σz ⨂ σz 
 (0.0086+0j)	*	 σy ⨂ I ⨂ σy 
 (0.0086+0j)	*	 σy ⨂ σy ⨂ I 
 (-1.43225+0j)	*	 σz ⨂ I ⨂ I 
 (-0.0086+0j)	*	 σz ⨂ σx ⨂ σz 
 (-0.0086+0j)	*	 σz ⨂ σz ⨂ σx 
 (-0.47742+0j)	*	 σz ⨂ σz ⨂ σz 


In [188]:
np.round(np.real(H_eff),2)

array([[0.0000, 0.0000, 0.0000, 0.0000, -0.0200, -0.0200, -0.0200, 0.0000],
       [0.0000, 1.9100, 0.0200, 0.0200, 0.0000, 0.0000, 0.0000, -0.0200],
       [0.0000, 0.0200, 1.9100, 0.0200, 0.0000, 0.0000, 0.0000, -0.0200],
       [0.0000, 0.0200, 0.0200, 1.9100, 0.0000, 0.0000, 0.0000, -0.0200],
       [-0.0200, 0.0000, 0.0000, 0.0000, 3.8200, 0.0200, 0.0200, 0.0000],
       [-0.0200, 0.0000, 0.0000, 0.0000, 0.0200, 3.8200, 0.0200, 0.0000],
       [-0.0200, 0.0000, 0.0000, 0.0000, 0.0200, 0.0200, 3.8200, 0.0000],
       [0.0000, -0.0200, -0.0200, -0.0200, 0.0000, 0.0000, 0.0000, 5.7300]])